# Install project dependencies

This cell will install the dependencies listed in `notebooks/requirements.txt` into the current Python environment used by this notebook. It uses the notebook's Python executable to ensure packages install into the correct environment.

Run the next cell to perform the installation.

In [3]:
try:
    # prefer parent path so notebook can be run from this folder
    %pip install -r ../requirements.txt
except Exception:
    # fallback to local path
    %pip install -r requirements.txt

print("Installation command executed. Restart kernel if necessary.")

Note: you may need to restart the kernel to use updated packages.
Installation command executed. Restart kernel if necessary.


# Share components, environments and models across workspaces

This is the companion notebook for the article on sharing components, environments and models across workspaces: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries 

### Prerequisites
Review the prerequisites section in the article: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries?tabs=python#prerequisites. To summarize, in addition to the Python SDK, you need an AzureML registry and an AzureML workspace in a region that is supported by the workspace.


### Scenarios

There are two scenarios where you'd want to use the same set of models, components and environments in multiple workspaces:

* Cross-workspace MLOps: You're training a model in a dev workspace and need to deploy it to test and prod workspaces. In this case you, want to have end-to-end lineage between endpoints to which the model is deployed in test or prod workspaces and the training job, metrics, code, data and environment that was used to train the model in the dev workspace.
* Share and reuse models and pipelines across different teams: Sharing and reuse improve collaboration and productivity. In this scenario, you may want to publish a trained model and the associated components and environments used to train it to a central catalog. From there, colleagues from other teams can search and reuse the assets you shared in their own experiments.

### Goals
* Create an environment and component in the registry.
* Use the component from registry to submit a model training job in a workspace.
* Register the trained model in the registry.
* Deploy the model from the registry to an online-endpoint in the workspace, then submit an inference request.


In [2]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import (
    Environment,
    BuildContext,
    Model,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes
import time, datetime, os
from dotenv import load_dotenv

load_dotenv(override=True)

SUBSCRIPTION_ID = os.environ["SUBSCRIPTION_ID"]
RESOURCE_GROUP = os.environ["DEV_RESOURCE_GROUP"]
AML_WORKSPACE_NAME = os.environ["DEV_WORKSPACE_NAME"]
EXTERNAL_REGISTRY_NAME = os.environ["DEV_REGISTRY_NAME"]
#USER_ASSIGNED_IDENTITY_NAME = os.environ["USER_ASSIGNED_IDENTITY_NAME"]
#USER_ASSIGNED_IDENTITY_RESOURCE_GROUP = os.environ["USER_ASSIGNED_IDENTITY_RESOURCE_GROUP"]
#USER_ASSIGNED_IDENTITY_CLIENT_ID = os.environ["USER_ASSIGNED_IDENTITY_CLIENT_ID"]
# print the sdk version - you many want to share this in the issue you will report if parts of this notebook don't work
#!pip show azure-ai-ml

### Setup authentication

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## Connect to a workspace and registry

Most samples create one client to connect to the workspace. However, in this sample, you need two clients. First client, called `ml_client_workspace`, will be used to connect to a workspace and run jobs or deploy endpoints. Second client, called `ml_client_registry` will be used to connect to the registry to create components, environments and models.

Replace the following:
* `<SUBSCRIPTION_ID>`
* `<RESOURCE_GROUP>`
* `<AML_WORKSPACE_NAME>`
* `<REGISTRY_NAME>`
 

In [4]:
ml_client_workspace = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=AML_WORKSPACE_NAME,
)
print(ml_client_workspace)

ml_client_registry = MLClient(credential=credential, registry_name=EXTERNAL_REGISTRY_NAME)
print(ml_client_registry)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000016306373620>,
         subscription_id=5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25,
         resource_group_name=rg-aml-ws-dev-cc-01,
         workspace_name=mlwdevcc01)


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000016306373620>,
         subscription_id=5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25,
         resource_group_name=rg-aml-reg-dev-cc-01,
         workspace_name=None)


### Create a version number and setup root directory 
Make sure that you set the version number to something unique if this notebook has been run before. You can use the timestamp to generate a unique version number, the sample code for which is commented out. This will prevent any name and version conflicts when creating assets.

Set the root directory in which the YAML definitions of the components, environments, etc. are present.

In [5]:
import time
import sys

# version = str(123456)
version = str(int(time.time()))
print("version: ", version)

version:  1762524483


### Create model in registry

You will now obtain the model trained by the pipeline job in the above step and create the model in the registry. For completeness, we are showing two options here.
* First option shows to create a model in registry from job output without downloading it. This option is recommended when you want to track the lineage between the training job and the model. You will create a model directly from the job output (without downloading it) in your workspace and then copy the model from workspace to registry. 
* Second option shows how to create a model in registry from local files. In this case you will download the model from the job output. This option is helpful if you have an existing model from some external source and want to host it in the registry to be shared with many workspaces.

Review this notebook to learn the different model types and how to create them in a workspace: [../../assets/model/model.ipynb](../model/model.ipynb). In the below example, you will work with a `mlflow_model` that will help you deploy this model for inference without writing any scoring scripts.


### Create model from local files 
Step b: Create a model in registry from files in a local folder. Note that you use the `ml_client_registry` client to create the model in registry. The syntax for creating model in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

> **Warning:** If you have successfully created a model in registry in the previous steps, this section will fail as the model with the name and version will already exist. 

In [6]:
# this section is optional, will fail if this model name and version is already created in the registry in the previous steps
mlflow_model = Model(
    path="./artifacts/model/",
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model-external-registry",
    version= version,
    description="MLflow model created from local path",
)
print(mlflow_model)
#ml_client_workspace.models.create_or_update(mlflow_model)
ml_client_registry.models.create_or_update(mlflow_model)

description: MLflow model created from local path
name: nyc-taxi-model-external-registry
path: C:\Users\jomedin\Documents\MLOPs-AzureML\notebooks\asset_sharing\artifacts\model
properties: {}
tags: {}
type: mlflow_model
version: '1762524483'



Subtype value SAS has no mapping, use base class DataReferenceCredentialDto.
Uploading model (0.0 MBs): 100%|##########| 2666/2666 [00:00<00:00, 17765.58it/s]




Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nyc-taxi-model-external-registry', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': 'azureml://registries/mlrdevcc01/models/nyc-taxi-model-external-registry/versions/1762524483', 'Resource__source_path': '', 'base_path': 'c:\\Users\\jomedin\\Documents\\MLOPs-AzureML\\notebooks\\asset_sharing', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000016308E58980>, 'serialize': <msrest.serialization.Serializer object at 0x0000016308D5FBD0>, 'version': '1762524483', 'latest_version': None, 'path': 'https://de270e222a9.blob.core.windows.net/mlrdevcc01-4d3344e6-400f-5679-847a-9a299c18322e/model', 'datastore': None, 'utc_time_created': None, 'flavors': {'python_function': {'env': 'conda.yaml', 'loader_module': 'mlflow.sklearn', '

### Create model in workspace

In [7]:
# this section is optional, will fail if this model name and version is already created in the registry in the previous steps
mlflow_model = Model(
    path="./artifacts/model/",
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model-ws",
    version= version,
    description="MLflow model created from local path",
)
print(mlflow_model)
ml_client_workspace.models.create_or_update(mlflow_model)

description: MLflow model created from local path
name: nyc-taxi-model-ws
path: C:\Users\jomedin\Documents\MLOPs-AzureML\notebooks\asset_sharing\artifacts\model
properties: {}
tags: {}
type: mlflow_model
version: '1762524483'



Uploading model (0.0 MBs): 100%|##########| 2666/2666 [00:00<00:00, 9711.47it/s]




Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nyc-taxi-model-ws', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/rg-aml-ws-dev-cc-01/providers/Microsoft.MachineLearningServices/workspaces/mlwdevcc01/models/nyc-taxi-model-ws/versions/1762524483', 'Resource__source_path': '', 'base_path': 'c:\\Users\\jomedin\\Documents\\MLOPs-AzureML\\notebooks\\asset_sharing', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000016308D607D0>, 'serialize': <msrest.serialization.Serializer object at 0x0000016308E1B4D0>, 'version': '1762524483', 'latest_version': None, 'path': 'azureml://subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/rg-aml-ws-dev-cc-01/workspaces/mlwdevcc01/datastores/workspaceblo

### Share model from workspace to registry

In [8]:
ml_client_workspace.models.share(
    name="nyc-taxi-model-ws",
    version=version,
    registry_name=EXTERNAL_REGISTRY_NAME,
    share_with_name="nyc-taxi-model-ws-ext",
    share_with_version=version,
)

Method share: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nyc-taxi-model-ws-ext', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': 'azureml://registries/mlrdevcc01/models/nyc-taxi-model-ws-ext/versions/1762524483', 'Resource__source_path': '', 'base_path': 'c:\\Users\\jomedin\\Documents\\MLOPs-AzureML\\notebooks\\asset_sharing', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000016308D551D0>, 'serialize': <msrest.serialization.Serializer object at 0x0000016308EECBB0>, 'version': '1762524483', 'latest_version': None, 'path': 'https://de270e222a9.blob.core.windows.net/mlrdevcc01-04c8e714-1cfc-5bda-a4fc-b6729bc77c96/LocalUpload/beaf4dff7bd507b2efc4d854506d9a608734143a0118f0b24568f96a7a57852a/model', 'datastore': None, 'utc_time_created': None, 'flavors': {'python_function': {'e

## Create environment in workspace and then share it to registry

### Create environment in workspace

In [ ]:
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="environment/online-endpoints-multimodel.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
# Capture the returned environment object to get the actual version
env_docker_conda = ml_client_workspace.environments.create_or_update(env_docker_conda)
print(f"Created environment version: {env_docker_conda.version}")

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/rg-aml-ws-dev-cc-01/providers/Microsoft.MachineLearningServices/workspaces/mlwdevcc01/environments/docker-image-plus-conda-example/versions/1', 'Resource__source_path': '', 'base_path': 'c:\\Users\\jomedin\\Documents\\MLOPs-AzureML\\notebooks\\asset_sharing', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000016308E07100>, 'serialize': <msrest.serialization.Serializer object at 0x0000016308F5F850>, 'version': '1', 'conda_file': {'chan

### Share environment to registry

In [13]:
ml_client_workspace.environments.share(
    name="docker-image-plus-conda-example",
    version=env_docker_conda.version,
    registry_name=EXTERNAL_REGISTRY_NAME,
    share_with_name="docker-image-plus-conda-example",
    share_with_version=env_docker_conda.version,
)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'default,latest,invisibleLatest'}, 'print_as_yaml': False, 'id': 'azureml://registries/mlrdevcc01/environments/docker-image-plus-conda-example/versions/1', 'Resource__source_path': '', 'base_path': 'c:\\Users\\jomedin\\Documents\\MLOPs-AzureML\\notebooks\\asset_sharing', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000001630A2F2520>, 'serialize': <msrest.serialization.Serializer object at 0x000001630A2D4910>, 'version': '1', 'conda_file': {'channels': ['defaults'], 'dependencies': ['python=3.8', 'pip', {'pip': ['pandas', 'numpy', 'scikit-le

### Deploy model from registry to online endpoint in workspace

You will deploy the model to an online endpoint and submit some sample inference requests in this section. Note that just like jobs, endpoints that host models are specific to a workspace. You can deploy the a model from a registry to many workspaces. This helps you develop a model in `dev` workspace, share it with a registry, and then deploy it to `test` and `prod` workspaces. 

### Get model from registry

Use the `ml_client_registry` client to get the model created in previous section from the registry. The syntax for creating component in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

In [14]:
mlflow_model_from_registry = ml_client_registry.models.get(
    name="nyc-taxi-model-ws-ext", version=version
)
print(mlflow_model_from_registry)

creation_context:
  created_at: '2025-11-07T14:09:39.129366+00:00'
  created_by: System Administrator
  created_by_type: User
  last_modified_at: '2025-11-07T14:10:09.874964+00:00'
  last_modified_by: System Administrator
  last_modified_by_type: User
description: MLflow model created from local path
flavors:
  python_function:
    env: conda.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.8.13
  sklearn:
    code: ''
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 0.24.1
id: azureml://registries/mlrdevcc01/models/nyc-taxi-model-ws-ext/versions/1762524483
name: nyc-taxi-model-ws-ext
path: https://de270e222a9.blob.core.windows.net/mlrdevcc01-04c8e714-1cfc-5bda-a4fc-b6729bc77c96/LocalUpload/beaf4dff7bd507b2efc4d854506d9a608734143a0118f0b24568f96a7a57852a/model
properties: {}
tags: {}
type: mlflow_model
version: '1762524483'



### Create an online endpoint 

Create Identity configuration for User Assigned Identity

In [17]:
# https://github.com/MicrosoftDocs/azure-ai-docs/blob/main/articles/machine-learning/how-to-identity-based-service-authentication.md
# https://learn.microsoft.com/en-us/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2&tabs=python#create-compute-with-managed-identity-to-access-docker-images-for-training

In [18]:
from azure.ai.ml.entities import IdentityConfiguration, ManagedIdentityConfiguration
from azure.ai.ml.constants import ManagedServiceIdentityType

uai = USER_ASSIGNED_IDENTITY_NAME  # replace with your user-assigned managed identity name
resource_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourcegroups/{USER_ASSIGNED_IDENTITY_RESOURCE_GROUP}/providers/Microsoft.ManagedIdentity/userAssignedIdentities/{uai}"
# Create an identity configuration from the user-assigned managed identity
managed_identity = ManagedIdentityConfiguration(resource_id=resource_id)
identity_config = IdentityConfiguration(type = ManagedServiceIdentityType.USER_ASSIGNED, user_assigned_identities=[managed_identity])

Create an online endpoint to deploy the model

In [19]:
online_endpoint_name = "endpoint-" + version
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    identity=identity_config,
)
ml_client_workspace.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-1752725482.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-1752725482.eastus2.inference.ml.azure.com/swagger.json', 'name': 'endpoint-1752725482', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {}, 'properties': {'createdBy': '7eb0cccd-2e21-4ca5-abb7-c65071ac0203', 'createdAt': '2025-07-17T04:12:08.018487+0000', 'lastModifiedAt': '2025-07-17T04:12:08.018487+0000', 'azureml.onlineendpointid': '/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourcegroups/publicazureml/providers/microsoft.machinelearningservices/workspaces/pazuremlregistrytest/onlineendpoints/endpoint-1752725482', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:850d046a-3781-47a9-86d9-5475e26d52

### Deploy the model from registry to the online endpoint

In [ ]:
# create a demo deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=mlflow_model_from_registry.id,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)
ml_client_workspace.online_deployments.begin_create_or_update(demo_deployment).result()

endpoint.traffic = {"demo": 100}
ml_client_workspace.begin_create_or_update(endpoint).result()

Check: endpoint endpoint-1752725482 exists


...............................

### Test the deployment

This section needs a sample request file `scoring-data.json` which is available in the root directory initialized in the beginning of this notebook: [../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/scoring-data.json](../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/scoring-data.json)

In [ ]:
# test the  deployment with some sample data
ml_client_workspace.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="scoring-data.json",
)

'[5.936981839396367, 21.51612674922012, 55.471569384161285]'

### Clean up resources

#### delete online endpoint 

In [ ]:
print(f"online_endpoint_name: {online_endpoint_name}")
ml_client_workspace.online_endpoints.begin_delete(name=online_endpoint_name).result()

online_endpoint_name: endpoint-1752701048
....................